In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

customers_url = "https://drive.google.com/uc?id=1bu_--mo79VdUG9oin4ybfFGRUSXAe-WE"
products_url = "https://drive.google.com/uc?id=1IKuDizVapw-hyktwfpoAoaGtHtTNHfd0"
transactions_url = "https://drive.google.com/uc?id=1saEqdbBB-vuk2hxoAf4TzDEsykdKlzbF"

customers_df = pd.read_csv(customers_url)
products_df = pd.read_csv(products_url)
transactions_df = pd.read_csv(transactions_url)

transactions_merged = transactions_df.merge(customers_df, on='CustomerID').merge(products_df, on='ProductID')
customer_product_matrix = pd.pivot_table(transactions_merged, values='Quantity', index='CustomerID', columns='ProductID', aggfunc=np.sum, fill_value=0)


<ipython-input-6-d8c02640fee2>:15: FutureWarning: The provided callable <function sum at 0x7ec4453cbe20> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  customer_product_matrix = pd.pivot_table(transactions_merged, values='Quantity', index='CustomerID', columns='ProductID', aggfunc=np.sum, fill_value=0)


In [14]:
scaler = StandardScaler()
customer_product_matrix_scaled = scaler.fit_transform(customer_product_matrix)
similarity_matrix = cosine_similarity(customer_product_matrix_scaled)

lookalikes = {}
for customer_id in range(1, 21):  # CustomerID C0001 to C0020
    customer_index = customer_id - 1
    similarity_scores = similarity_matrix[customer_index]
    similar_customers = np.argsort(similarity_scores)[-4:-1]  # Top 3 excluding itself
    similar_scores = similarity_scores[similar_customers]
    lookalikes[f'C{customer_id:04d}'] = [(f'C{similar_customers[i]+1:04d}', similar_scores[i]) for i in range(3)]

In [15]:
lookalike_data = []
for customer_id, similar_customers in lookalikes.items():
    for similar_customer in similar_customers:
        lookalike_data.append([customer_id, similar_customer[0], similar_customer[1]])

lookalike_df = pd.DataFrame(lookalike_data, columns=['CustomerID', 'LookalikeCustomerID', 'SimilarityScore'])
lookalike_df.to_csv('Maithreyi_A_LookAlike.csv', index=False)
print(lookalike_df.head())

  CustomerID LookalikeCustomerID  SimilarityScore
0      C0001               C0020         0.366609
1      C0001               C0104         0.374002
2      C0001               C0193         0.404928
3      C0002               C0071         0.320158
4      C0002               C0091         0.383778
